In [3]:
from __future__ import print_function
from IPython.display import clear_output
    
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [5]:
def train(LOG_INTERVAL, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            #clear_output(wait=True)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
# Training settings
BATCH_SIZE = 64
TEST_BATCH_SIZE=1000
EPOCHS=2
LR=1.0
GAMMA=0.7
SEED=1
LOG_INTERVAL=10

use_cuda = torch.cuda.is_available()
torch.manual_seed(SEED)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': BATCH_SIZE}
test_kwargs = {'batch_size': TEST_BATCH_SIZE}

if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

dataset1 = datasets.MNIST('../data', train=True, transform=transform)

dataset2 = datasets.MNIST('../data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=LR)

scheduler = StepLR(optimizer, step_size=1, gamma=GAMMA)
for epoch in range(1, EPOCHS + 1):
    train(LOG_INTERVAL, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn_1.pt")

c:\users\mmoustafa\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.283075
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.869621
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.914569
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.714923
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.450578
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.461104
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.137054
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.487427
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.275324
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.156311
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.240491
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.202436
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.323166
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.161264
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.347525
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.156192
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.239867
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.184010
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.257274
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.123487
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.118463
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.146364
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.092210
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.143529
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.149105
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.025869
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.023205
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.006429
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.039608
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.017107
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.102549
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.255641
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.141495
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.008090
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.005914
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.136195
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.011956
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.068452
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.090311
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.017930


In [28]:
for name, param in model.named_parameters():
    if "conv" in name:
        if "weight" in name:
            print(name)
            print(param.shape)
        else:
            print(name)
            print(param)
    else:
        if "weight" in name:
            print(name)
            print(param.shape)
        else:
            print(name)
            print(param)

conv1.weight
torch.Size([32, 1, 3, 3])
conv1.bias
Parameter containing:
tensor([-0.1155, -0.0570, -0.0681, -0.1644,  0.1017,  0.0468, -0.1271, -0.1192,
        -0.1268,  0.0304, -0.1567, -0.2425, -0.1611, -0.1287, -0.0784, -0.1177,
        -0.2234, -0.1681, -0.1877,  0.0471, -0.0552,  0.1628, -0.1160, -0.1477,
        -0.2545, -0.2291,  0.0618, -0.2541, -0.2045,  0.0234,  0.1682, -0.0538],
       device='cuda:0', requires_grad=True)
conv2.weight
torch.Size([64, 32, 3, 3])
conv2.bias
Parameter containing:
tensor([-0.0511,  0.0099, -0.0848, -0.0236,  0.0315, -0.0072,  0.0516, -0.0621,
         0.0566, -0.0666,  0.0038,  0.0059,  0.0340, -0.1155,  0.0281,  0.0377,
         0.0302, -0.0089, -0.0490, -0.0151, -0.0143,  0.0054, -0.0540, -0.0301,
         0.0028,  0.0135,  0.0380,  0.0341, -0.0703, -0.0811,  0.0197, -0.0372,
         0.0273,  0.0271,  0.0091,  0.0176, -0.0355, -0.0666,  0.0096,  0.0036,
         0.0444, -0.0598, -0.0439, -0.0252,  0.0615, -0.0632, -0.0501, -0.0640,
        -0